# Cauchy example

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

plt.rcParams.update({'figure.max_open_warning': 0})
plt.rcParams["figure.figsize"]=15,7.5
plt.rcParams.update({'font.size': 24})

In [24]:
# target definition
def lp(x): return -np.log(1+x**2)-np.log(np.pi)
def randp(size): return np.random.standard_cauchy(size)

In [19]:
# gaussian variational approximation
def lq(x,mu,logsigma):
    sigma=np.exp(logsigma)
    return -0.5*((x-mu)/sigma)**2-0.5*np.log(2*np.pi*sigma**2)

In [33]:
# gradients

def mu_gradient(B,mu,lsigma,lp,randp,divergence='rev-kl'):
    if divergence=='rev-kl':
        sample=mu+np.exp(lsigma)*np.random.randn(B) #~N(mu,sigma^2)
        return np.mean((lq(sample,mu,lsigma)-lp(sample))*(sample-mu)*np.exp(-2*lsigma))
    return

def ls_gradient(B,mu,lsigma,lp,randp,divergence='rev-kl'):
    if divergence=='rev-kl':
        sample=mu+np.exp(lsigma)*np.random.randn(B) #~N(mu,sigma^2)
        return np.mean((lq(sample,mu,lsigma)-lp(sample))*(np.exp(-2*lsigma)*(sample-mu)**2-1.))
    return

In [48]:
# optimization wrapper
def gaussianVI(lp,randp,divergence='rev-kl',B=1000,max_iters=1001,lr_mu=1e-2,lr_lsigma=1e-2):
    # initial values
    mus=np.zeros(max_iters+1)
    mus[0]=np.random.randn() # initialize mu at a draw from std normal (cause 0 is probably the opt, let's make it harder)
    lsigmas=np.zeros(max_iters+1)
    lsigmas[0]=np.random.randn() # ditto

    # optimize
    print('Initial μ: '+str(mus[0]))
    print('Initial σ: '+str(np.exp(lsigmas[0])))
    print()
    print('Iter  |          μ          |          σ          |      μ gradient     |   logσ gradient')
    for t in range(max_iters):
        # calculate gradients
        grad_mu=mu_gradient(B,mus[t],lsigmas[t],lp,randp,divergence)
        grad_lsigma=ls_gradient(B,mus[t],lsigmas[t],lp,randp,divergence)
    
        # take step
        mus[t+1]=mus[t]-lr_mu*grad_mu
        lsigmas[t+1]=lsigmas[t]-lr_lsigma*grad_lsigma
    
        # do printout
        if t%(max_iters//10)==0: print('  '+str(t)+'   | '+str(mus[t+1])+'   | '+str(np.exp(lsigmas[t+1]))+'   | '+str(grad_mu)+'  | '+str(grad_lsigma))
    # end for
    
    print()
    print('Final μ: '+str(mus[-1]))
    print('Final σ: '+str(np.exp(lsigmas[-1])))
    return mus,lsigmas

In [52]:
# optimization

# settings
np.random.seed(520)
max_iters=10001
lr_mu=1e-1
lr_lsigma=1e-1
B=10000 # monte carlo sample size for gradient estimation
divergence='rev-kl'

mus,lsigmas=gaussianVI(lp,randp,divergence,B,max_iters,lr_mu,lr_lsigma)

Initial μ: -1.4109450484350377
Initial σ: 0.7555405048862535

Iter  |          μ          |          σ          |      μ gradient     |   logσ gradient
  0   | -1.330051637102188   | 0.8273992143980493   | -0.8089341133284977  | -0.9085391091811823
  1000   | 0.0005270524291469854   | 1.6332116399996317   | -0.0013887432507014561  | -0.002826871874133091
  2000   | 0.0031146345768370423   | 1.6450606691249248   | 0.0005891705981535329  | 0.02277744353551756
  3000   | 0.0005259391525982489   | 1.6362632856691977   | 0.0007629950059253183  | 0.0024541545456553343
  4000   | -0.0008499687316928625   | 1.6406734335863147   | 0.0028831615818133284  | 0.012191286539415832
  5000   | -0.0006150767688435494   | 1.6374103119659678   | 0.0030046832948064993  | 0.024323820942381117
  6000   | 0.0006711642414678229   | 1.6467761628271074   | -0.0008050145988884044  | 0.004942309406694695
  7000   | -0.004359731588668625   | 1.6364947601606012   | 0.0021833479612577617  | -0.02994362784271031
  80